<a href="https://colab.research.google.com/github/VaibhavBhusawale/Predict-Product-Backorders/blob/main/Predict_Product_Backorders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)

In [2]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [4]:
Test = pd.read_csv('/content/drive/MyDrive/backorder_Test_Dataset_v2.csv')
Train = pd.read_csv('/content/drive/MyDrive/backorder_Training_Dataset_v2.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
Test.head()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,3285085,62.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-99.00,-99.00,0.0,Yes,No,No,Yes,No,No
1,3285131,9.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-99.00,-99.00,0.0,No,No,Yes,No,No,No
2,3285358,17.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.92,0.95,0.0,No,No,No,Yes,No,No
3,3285517,9.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.78,0.75,0.0,No,No,Yes,Yes,No,No
4,3285608,2.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.54,0.71,0.0,No,No,No,Yes,No,No


In [7]:
Train.head()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,1026827,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-99.00,-99.00,0.0,No,No,No,Yes,No,No
1,1043384,2.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.99,0.99,0.0,No,No,No,Yes,No,No
2,1043696,2.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-99.00,-99.00,0.0,Yes,No,No,Yes,No,No
3,1043852,7.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.10,0.13,0.0,No,No,No,Yes,No,No
4,1044048,8.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-99.00,-99.00,0.0,Yes,No,No,Yes,No,No


In [10]:
Test.shape

(242076, 23)

In [12]:
Test.columns


Index(['sku', 'national_inv', 'lead_time', 'in_transit_qty',
       'forecast_3_month', 'forecast_6_month', 'forecast_9_month',
       'sales_1_month', 'sales_3_month', 'sales_6_month', 'sales_9_month',
       'min_bank', 'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
       'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder'],
      dtype='object')

In [9]:
Test.isnull().sum()

sku                      0
national_inv             1
lead_time            14725
in_transit_qty           1
forecast_3_month         1
forecast_6_month         1
forecast_9_month         1
sales_1_month            1
sales_3_month            1
sales_6_month            1
sales_9_month            1
min_bank                 1
potential_issue          1
pieces_past_due          1
perf_6_month_avg         1
perf_12_month_avg        1
local_bo_qty             1
deck_risk                1
oe_constraint            1
ppap_risk                1
stop_auto_buy            1
rev_stop                 1
went_on_backorder        1
dtype: int64

In [19]:
Test.dropna(inplace=True)

In [18]:
Test.duplicated().sum()

0

In [21]:
Test.describe()

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty
count,2.273510e+05,227351.000000,227351.000000,2.273510e+05,2.273510e+05,2.273510e+05,227351.000000,2.273510e+05,2.273510e+05,2.273510e+05,227351.000000,227351.000000,227351.000000,227351.000000,227351.000000
mean,4.947719e+02,7.923018,36.922037,1.924650e+02,3.697318e+02,5.388228e+02,52.262159,1.749552e+02,3.492733e+02,5.284617e+02,53.315732,1.941249,-1.141629,-0.650418,0.886326
std,3.014668e+04,7.041410,741.337000,5.828062e+03,1.040103e+04,1.455610e+04,1413.910573,4.798854e+03,9.227229e+03,1.410395e+04,1152.838833,184.373362,13.712759,11.847950,47.004905
min,-2.541400e+04,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,-99.000000,-99.000000,0.000000
25%,4.000000e+00,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.700000,0.680000,0.000000
50%,1.500000e+01,8.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.000000e+00,3.000000e+00,4.000000e+00,0.000000,0.000000,0.840000,0.820000,0.000000
75%,7.900000e+01,9.000000,0.000000,5.000000e+00,1.600000e+01,2.600000e+01,4.000000,1.500000e+01,3.300000e+01,5.000000e+01,4.000000,0.000000,0.970000,0.960000,0.000000
max,1.214579e+07,52.000000,186624.000000,1.510592e+06,2.157024e+06,3.162260e+06,299190.000000,1.086554e+06,2.103389e+06,3.195211e+06,196869.000000,79964.000000,1.000000,1.000000,6232.000000


In [20]:
Test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227351 entries, 2 to 242074
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   sku                227351 non-null  object 
 1   national_inv       227351 non-null  float64
 2   lead_time          227351 non-null  float64
 3   in_transit_qty     227351 non-null  float64
 4   forecast_3_month   227351 non-null  float64
 5   forecast_6_month   227351 non-null  float64
 6   forecast_9_month   227351 non-null  float64
 7   sales_1_month      227351 non-null  float64
 8   sales_3_month      227351 non-null  float64
 9   sales_6_month      227351 non-null  float64
 10  sales_9_month      227351 non-null  float64
 11  min_bank           227351 non-null  float64
 12  potential_issue    227351 non-null  object 
 13  pieces_past_due    227351 non-null  float64
 14  perf_6_month_avg   227351 non-null  float64
 15  perf_12_month_avg  227351 non-null  float64
 16  lo

In [23]:
Test.head()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
2,3285358,17.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.92,0.95,0.0,No,No,No,Yes,No,No
3,3285517,9.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.78,0.75,0.0,No,No,Yes,Yes,No,No
4,3285608,2.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.54,0.71,0.0,No,No,No,Yes,No,No
5,3285960,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.37,0.68,0.0,No,No,No,Yes,No,No
13,3288094,208.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.66,0.64,0.0,No,No,No,Yes,No,No
